In [14]:
import networkx as nx
import numpy as np
from pathlib import Path
import copy
import pandas as pd
import random

from BNS_JT import trans, branch, variable, cpm, brc, config

HOME = Path.cwd()
random.seed(1)

# EMA

Randomly divide edges into three groups with different failure probabilities for demonstration's sake.

In [15]:
cfg = config.Config('C:/Users/jb622s/git/BNS-JT/BNS_JT/demos/ema/config.json')
st_br_to_cs = {'f':0, 's':1, 'u': 2}

n_edge = len(cfg.infra['edges'])

integers = list(range(n_edge))
random.shuffle(integers)
group_size = len(integers) // 3

prob_groups = [sorted(integers[:group_size]), sorted(integers[group_size:2*group_size]), sorted(integers[2*group_size:])]
probs_setting = [{0:0.01, 1: 0.04, 2: 0.95}, {0:0.03, 1: 0.12, 2: 0.85}, {0:0.06, 1: 0.24, 2: 0.70}]

probs = {}
for i in range(n_edge):
    g_idx = next(index for index, group in enumerate(prob_groups) if i in group)
    probs['e'+str(i+1)] = probs_setting[g_idx]

print(probs)

{'e1': {0: 0.06, 1: 0.24, 2: 0.7}, 'e2': {0: 0.06, 1: 0.24, 2: 0.7}, 'e3': {0: 0.06, 1: 0.24, 2: 0.7}, 'e4': {0: 0.06, 1: 0.24, 2: 0.7}, 'e5': {0: 0.01, 1: 0.04, 2: 0.95}, 'e6': {0: 0.01, 1: 0.04, 2: 0.95}, 'e7': {0: 0.01, 1: 0.04, 2: 0.95}, 'e8': {0: 0.01, 1: 0.04, 2: 0.95}, 'e9': {0: 0.01, 1: 0.04, 2: 0.95}, 'e10': {0: 0.01, 1: 0.04, 2: 0.95}, 'e11': {0: 0.01, 1: 0.04, 2: 0.95}, 'e12': {0: 0.03, 1: 0.12, 2: 0.85}, 'e13': {0: 0.06, 1: 0.24, 2: 0.7}, 'e14': {0: 0.06, 1: 0.24, 2: 0.7}, 'e15': {0: 0.01, 1: 0.04, 2: 0.95}, 'e16': {0: 0.06, 1: 0.24, 2: 0.7}, 'e17': {0: 0.06, 1: 0.24, 2: 0.7}, 'e18': {0: 0.01, 1: 0.04, 2: 0.95}, 'e19': {0: 0.03, 1: 0.12, 2: 0.85}, 'e20': {0: 0.01, 1: 0.04, 2: 0.95}, 'e21': {0: 0.01, 1: 0.04, 2: 0.95}, 'e22': {0: 0.01, 1: 0.04, 2: 0.95}, 'e23': {0: 0.01, 1: 0.04, 2: 0.95}, 'e24': {0: 0.03, 1: 0.12, 2: 0.85}, 'e25': {0: 0.03, 1: 0.12, 2: 0.85}, 'e26': {0: 0.03, 1: 0.12, 2: 0.85}, 'e27': {0: 0.06, 1: 0.24, 2: 0.7}, 'e28': {0: 0.06, 1: 0.24, 2: 0.7}, 'e29': {0:

In [16]:
print(cfg.infra['edges'])
print(n_edge)

G = nx.Graph()
for e, v in cfg.infra['edges'].items():
    G.add_edge(v['origin'], v['destination'], label=e, weight=v['weight'])

{'e1': {'origin': 'n1', 'destination': 'n3', 'link_capacity': None, 'weight': 0.1161234007510975}, 'e2': {'origin': 'n1', 'destination': 'n7', 'link_capacity': None, 'weight': 0.1166500015002143}, 'e3': {'origin': 'n1', 'destination': 'n9', 'link_capacity': None, 'weight': 0.14076883621739575}, 'e4': {'origin': 'n2', 'destination': 'n3', 'link_capacity': None, 'weight': 0.04063165319058527}, 'e5': {'origin': 'n3', 'destination': 'n6', 'link_capacity': None, 'weight': 0.06334335846006278}, 'e6': {'origin': 'n3', 'destination': 'n7', 'link_capacity': None, 'weight': 0.0828644294736892}, 'e7': {'origin': 'n4', 'destination': 'n6', 'link_capacity': None, 'weight': 0.0400481841411069}, 'e8': {'origin': 'n4', 'destination': 'n8', 'link_capacity': None, 'weight': 0.0471304521514488}, 'e9': {'origin': 'n5', 'destination': 'n10', 'link_capacity': None, 'weight': 0.0755459597397504}, 'e10': {'origin': 'n5', 'destination': 'n11', 'link_capacity': None, 'weight': 0.08307627628270292}, 'e11': {'ori

In [17]:
varis = {}
cpms = {}
for k in cfg.infra['edges'].keys():
    varis[k] = variable.Variable(name=k, values = cfg.scenarios['scenarios']['s1'][k])
    cpms[k] = cpm.Cpm(variables = [varis[k]], no_child=1,
                        C = np.array([0, 1, 2]).T, p = [probs[k][0], probs[k][1], probs[k][2]])

thres = 2
st_br_to_cs = {'f': 0, 's': 1, 'u': 2}

#csys_by_od, varis_by_od = {}, {}
# branches by od_pair
#for k, od_pair in cfg.infra['ODs'].items():
comps_st_itc = {k: len(v.values) - 1 for k, v in varis.items()}
#od_pair = ('n1', 'n14')
od_pair = ('n1','n53')
d_time_itc, path_itc, path_itc_e = trans.get_time_and_path_given_comps(comps_st_itc, G, od_pair, varis)

# system function
sys_fun = trans.sys_fun_wrap(G, od_pair, varis=varis, thres=thres * d_time_itc)

In [18]:
brs, rules, sys_res, monitor = brc.run(varis, probs, sys_fun, max_sf=50, display_freq=25)

[System function runs 25]..
The # of found non-dominated rules (f, s): 20 (0, 20)
Probability of branchs (f, s, u): (0.0000e+00, 7.23e-01, 2.7669e-01)
The # of branches (f, s, u), (min, avg) len of rf: 953 (0, 171, 782), (0, 0.00)
Elapsed seconds (average per round): 1.11e+01 (4.44e-01)
[System function runs 50]..
The # of found non-dominated rules (f, s): 38 (0, 38)
Probability of branchs (f, s, u): (0.0000e+00, 8.26e-01, 1.7393e-01)
The # of branches (f, s, u), (min, avg) len of rf: 3689 (0, 784, 2905), (0, 0.00)
Elapsed seconds (average per round): 7.65e+01 (1.53e+00)

*Final decomposition is completed with 2173 branches (originally 3689 branches).

***Analysis completed with f_sys runs 50: out_flag = max_sf***
The # of found non-dominated rules (f, s): 38 (0, 38)
Probability of branchs (f, s, u): (0.0000e+00, 8.28e-01, 1.7244e-01)
The # of branches (f, s, u), (min, avg) len of rf: 2173 (0, 484, 1689), (0, 0.00)
Elapsed seconds (average per round): 8.98e+01 (1.76e+00)


In [19]:
print(f"Survival rules: {rules['s']}")
print(f"Failure rules: {rules['f']}")


Survival rules: [{'e3': 2, 'e19': 2, 'e24': 1, 'e27': 2, 'e44': 2, 'e81': 2, 'e98': 2}, {'e3': 2, 'e19': 2, 'e24': 2, 'e26': 2, 'e31': 2, 'e42': 2, 'e56': 2, 'e75': 2, 'e84': 2, 'e89': 2, 'e92': 2, 'e107': 2}, {'e3': 2, 'e19': 2, 'e24': 2, 'e27': 2, 'e44': 2, 'e81': 2, 'e97': 2, 'e106': 2}, {'e1': 2, 'e5': 2, 'e13': 2, 'e31': 2, 'e42': 2, 'e56': 2, 'e75': 2, 'e84': 2, 'e89': 2, 'e92': 2, 'e107': 2}, {'e2': 2, 'e15': 2, 'e24': 2, 'e27': 2, 'e44': 2, 'e81': 2, 'e97': 2, 'e106': 2}, {'e3': 2, 'e19': 2, 'e24': 2, 'e26': 2, 'e31': 2, 'e44': 2, 'e81': 2, 'e97': 2, 'e106': 2}, {'e3': 2, 'e19': 2, 'e24': 2, 'e26': 1, 'e31': 2, 'e44': 2, 'e81': 2, 'e98': 2}, {'e3': 2, 'e19': 2, 'e24': 1, 'e26': 2, 'e31': 2, 'e44': 2, 'e81': 2, 'e98': 2}, {'e3': 2, 'e19': 2, 'e24': 2, 'e26': 2, 'e28': 1, 'e30': 2, 'e44': 2, 'e81': 2, 'e98': 2}, {'e3': 1, 'e19': 2, 'e24': 2, 'e27': 2, 'e44': 2, 'e81': 2, 'e98': 2}, {'e2': 2, 'e6': 2, 'e5': 2, 'e13': 2, 'e31': 2, 'e44': 2, 'e81': 2, 'e98': 2}, {'e2': 1, 'e15': 2, 